In [2]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.9 MB/s eta 0:00:0000:01


In [3]:
from importlib.metadata import version

print("torch version:", version("torch"))
print("tiktoken version:", version("tiktoken"))

torch version: 2.1.2+cpu
tiktoken version: 0.7.0


In [4]:
import urllib

# URL of the PDF file
url = "https://ag.gov.np/files/Constitution-of-Nepal_2072_Eng_www.moljpa.gov_.npDate-72_11_16.pdf"

# Open the URL and download the PDF
response = urllib.request.urlopen(url)
pdf_data = response.read()

# Save the PDF to a local file
with open("Constitution-of-Nepal_2072_Eng.pdf", "wb") as f:
    f.write(pdf_data)

print("PDF downloaded and saved as 'Constitution-of-Nepal_2072_Eng.pdf'")


PDF downloaded and saved as 'Constitution-of-Nepal_2072_Eng.pdf'


In [5]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.9 MB/s eta 0:00:00ta 0:00:01


In [6]:
import PyPDF2

# Open the downloaded PDF file
with open("Constitution-of-Nepal_2072_Eng.pdf", "rb") as file:
    # Initialize a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)
    
    # Extract text from each page of the PDF
    raw_text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        raw_text += page.extract_text()

# Print the first 500 characters to preview the content
print(raw_text[:50])


 
1 
  
 
 
 
THE CONSTITUTION OF NEPAL  
   
2 
 


In [7]:
print(len(raw_text))

344782


**Split text into tokens**

In [8]:
import re
preprocessed = re.split(r'([,.:;?_!-"()\']|--|\s|[@#%&])', raw_text)

preprocessed = [item.strip() for item in preprocessed if item.strip()]
print(preprocessed[:30])

['1', 'THE', 'CONSTITUTION', 'OF', 'NEPAL', '2', 'Table', 'of', 'Content', 's', 'Preamble', 'Part-1', 'Preliminary', 'Part-2', 'Citizenship', 'Part-3', 'Fundamental', 'Rights', 'and', 'Duties', 'Part-4', 'Directive', 'Principles', ',', 'Policies', 'and', 'Obligations', 'of', 'the', 'State']


In [9]:
print(len(preprocessed))

65101


In [10]:
#Unique tokens
len(set(preprocessed))

4525

**Create a vocabulary**

In [11]:
all_words = sorted(set(preprocessed))
vocab_size = len(all_words)

print(vocab_size)

4525


In [12]:
#Give indexes and put that as dictionary
vocab = {token:integer for integer,token in enumerate(all_words)}

# Simple Tokenization of text****

In [13]:
"""Putting all together"""

class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}
    
    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
                                
        preprocessed = [
            item.strip() for item in preprocessed if item.strip()
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids
        
    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.?!"()\'])', r'\1', text)
        return text

In [14]:
tokenizer = SimpleTokenizerV1(vocab)
tokenizer.encode(""" """)


[]

# Using Bytepair Encoding 

In [15]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.7.0


In [16]:
tokenizer = tiktoken.get_encoding("gpt2")

In [17]:
text = (
    "No law shall be made providing for the death penalty to any one<|endoftext|> Every citizen"
     "of"
)

integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})

print(integers)

[2949, 1099, 2236, 307, 925, 4955, 329, 262, 1918, 7389, 284, 597, 530, 50256, 3887, 9511, 1659]


In [18]:
tokenizer.encode("fddfsdfsf")

[69, 1860, 9501, 7568, 28202]

In [19]:
tokenizer.decode([1860])

'dd'

**Data Sampling with a sliding window**

**We train LLMs to generate one word at a time, so we want to prepare the training data accordingly where the next word in a sequence represents the target to predict**

In [20]:
import PyPDF2
# Open the downloaded PDF file
with open("Constitution-of-Nepal_2072_Eng.pdf", "rb") as file:
    # Initialize a PDF reader object
    pdf_reader = PyPDF2.PdfReader(file)
    
    # Extract text from each page of the PDF
    raw_text = ""
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        raw_text += page.extract_text()
enc_text = tokenizer.encode(raw_text)
print(len(enc_text))

86531


In [30]:

vocab_size = len(set(enc_text))
print(vocab_size)

5305


In [22]:
import torch
from torch.utils.data import Dataset, DataLoader


class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        # Tokenize the entire text
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Use a sliding window to chunk the book into overlapping sequences of max_length
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]
            target_chunk = token_ids[i + 1: i + max_length + 1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]

In [23]:
def create_dataloader_v1(txt, batch_size=4, max_length=256, 
                         stride=128, shuffle=True, drop_last=True,
                         num_workers=0):

    # Initialize the tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create dataset
    dataset = GPTDatasetV1(txt, tokenizer, max_length, stride)

    # Create dataloader
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

    return dataloader

**Test a dataloader with a batch size of 1 and context size of 4**

In [24]:
dataloader = create_dataloader_v1(
    raw_text, batch_size=1, max_length=4, stride=1, shuffle=False
)

data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[220, 198,  16, 220]]), tensor([[198,  16, 220, 198]])]


In [25]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[198,  16, 220, 198]]), tensor([[ 16, 220, 198, 220]])]


In [26]:
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)

data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Inputs:\n", inputs)
print("\nTargets:\n", targets)

Inputs:
 tensor([[  220,   198,    16,   220],
        [  198,   220,   220,   198],
        [  220,   198,   220,   198],
        [  220,   198, 10970,  7102],
        [ 2257,  2043, 35354,  3963],
        [  399,  8905,  1847,   220],
        [  220,   198,   220,   220],
        [  220,   198,    17,   220]])

Targets:
 tensor([[  198,    16,   220,   198],
        [  220,   220,   198,   220],
        [  198,   220,   198,   220],
        [  198, 10970,  7102,  2257],
        [ 2043, 35354,  3963,   399],
        [ 8905,  1847,   220,   220],
        [  198,   220,   220,   220],
        [  198,    17,   220,   198]])


**Create token embeddings**

# Coding an LLM Architecture

In [27]:
GPT_CONFIG_124M = {
    "vocab_size": 5305,    # Vocabulary size from BPE
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.0,       # Dropout rate
    "qkv_bias": True       # Query-Key-Value bias
}

**Coding the GPT Model**

In [32]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

In [35]:
print(inputs)

tensor([[  220,   198,    16,   220],
        [  198,   220,   220,   198],
        [  220,   198,   220,   198],
        [  220,   198, 10970,  7102],
        [ 2257,  2043, 35354,  3963],
        [  399,  8905,  1847,   220],
        [  220,   198,   220,   220],
        [  220,   198,    17,   220]])


In [44]:
import torch
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        # Ensure the input tensor is in float32 format
        if x.dtype != torch.float32:
            x = x.float()

        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        
        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec
torch.manual_seed(123)
# Create the SelfAttention_v1 instance
sa_v1 = SelfAttention_v1(d_in, d_out)
# Forward pass
print(sa_v1(inputs))


tensor([[158.8819, 150.6755, 153.9783, 179.2892, 164.8145, 154.0112, 148.9656,
         161.0673, 156.5470, 171.6551, 151.0642, 195.4609, 135.7174, 169.4190,
         167.8739, 149.3526, 162.2017, 167.2431, 157.0390, 157.4042, 157.0929,
         180.7577, 130.4422, 155.6458, 152.1001, 176.0182, 166.5292, 163.6009,
         151.5780, 161.1508, 156.4633, 166.1018],
        [158.8819, 150.6755, 153.9783, 179.2892, 164.8145, 154.0112, 148.9656,
         161.0673, 156.5470, 171.6551, 151.0642, 195.4609, 135.7174, 169.4190,
         167.8739, 149.3526, 162.2017, 167.2431, 157.0390, 157.4042, 157.0929,
         180.7577, 130.4422, 155.6458, 152.1001, 176.0182, 166.5292, 163.6009,
         151.5780, 161.1508, 156.4633, 166.1018],
        [158.8819, 150.6755, 153.9783, 179.2892, 164.8145, 154.0112, 148.9656,
         161.0673, 156.5470, 171.6551, 151.0642, 195.4609, 135.7174, 169.4190,
         167.8739, 149.3526, 162.2017, 167.2431, 157.0390, 157.4042, 157.0929,
         180.7577, 130.4422, 15

In [46]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        if x.dtype != torch.float32:
            x = x.float()

        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-3.2235, -3.7357,  3.6138, -6.4185, -0.0431, -0.1825, -2.7255,  3.2873,
         -1.2094,  0.0430,  3.2942,  1.0218, -0.9350, -1.0848, -7.7038, -0.3324,
          1.1618,  0.7010, -3.9985,  0.8794, -1.9087, -5.3913, -1.9001,  0.1609,
          0.4097, -2.7980, -1.4895, -0.7474,  1.6869,  4.2371, -1.5783,  0.0826],
        [-3.2234, -3.7357,  3.6137, -6.4184, -0.0430, -0.1826, -2.7255,  3.2872,
         -1.2094,  0.0431,  3.2941,  1.0218, -0.9349, -1.0848, -7.7039, -0.3325,
          1.1618,  0.7011, -3.9985,  0.8794, -1.9087, -5.3913, -1.9001,  0.1609,
          0.4097, -2.7979, -1.4896, -0.7473,  1.6869,  4.2371, -1.5783,  0.0826],
        [-3.2202, -3.7356,  3.6085, -6.4150, -0.0413, -0.1859, -2.7262,  3.2816,
         -1.2054,  0.0490,  3.2879,  1.0218, -0.9315, -1.0841, -7.7069, -0.3341,
          1.1612,  0.7050, -3.9960,  0.8781, -1.9092, -5.3891, -1.9025,  0.1632,
          0.4129, -2.7931, -1.4912, -0.7430,  1.6863,  4.2395, -1.5787,  0.0827],
        [-3.2223, -3.7353

In [53]:
print(type(sa_v2.W_query))

<class 'torch.nn.modules.linear.Linear'>


In [66]:
# Reuse the query and key weight matrices
inputs = inputs.float()
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[6.2818e-12, 4.5580e-09, 7.2956e-06, 9.9999e-01, 6.9826e-10],
        [2.6285e-08, 8.8171e-07, 3.4499e-05, 9.9996e-01, 5.2370e-07],
        [1.2302e-08, 1.1123e-04, 1.3304e-03, 9.9856e-01, 1.6148e-06],
        [7.6703e-08, 2.2564e-04, 2.9404e-04, 9.9945e-01, 2.7256e-05],
        [3.8708e-07, 3.5687e-07, 1.4203e-04, 9.9986e-01, 1.0779e-07]],
       grad_fn=<SoftmaxBackward0>)


In [67]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0.],
        [1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1.]])


In [69]:
print(attn_weights)

tensor([[6.2818e-12, 4.5580e-09, 7.2956e-06, 9.9999e-01, 6.9826e-10],
        [2.6285e-08, 8.8171e-07, 3.4499e-05, 9.9996e-01, 5.2370e-07],
        [1.2302e-08, 1.1123e-04, 1.3304e-03, 9.9856e-01, 1.6148e-06],
        [7.6703e-08, 2.2564e-04, 2.9404e-04, 9.9945e-01, 2.7256e-05],
        [3.8708e-07, 3.5687e-07, 1.4203e-04, 9.9986e-01, 1.0779e-07]],
       grad_fn=<SoftmaxBackward0>)


In [68]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[6.2818e-12, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.6285e-08, 8.8171e-07, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [1.2302e-08, 1.1123e-04, 1.3304e-03, 0.0000e+00, 0.0000e+00],
        [7.6703e-08, 2.2564e-04, 2.9404e-04, 9.9945e-01, 0.0000e+00],
        [3.8708e-07, 3.5687e-07, 1.4203e-04, 9.9986e-01, 1.0779e-07]],
       grad_fn=<MulBackward0>)


In [70]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [2.8948e-02, 9.7105e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00],
        [8.5336e-06, 7.7160e-02, 9.2283e-01, 0.0000e+00, 0.0000e+00],
        [7.6705e-08, 2.2565e-04, 2.9404e-04, 9.9948e-01, 0.0000e+00],
        [3.8708e-07, 3.5687e-07, 1.4203e-04, 9.9986e-01, 1.0779e-07]],
       grad_fn=<DivBackward0>)
